# Section 0: Import Packages

In [ ]:
# Data manipulation and visualisation
import pandas as pd                                                           # to deal with pandas dataframe
import numpy as np                                                            # to deal with numbers
import matplotlib.pyplot as plt                                               # to plot graphs

# Statistical analysis
from statsmodels.graphics import tsaplots                                     # to plot graphs
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf                 # to plot ACF and PACF graphs
from statsmodels.tsa.stattools import adfuller                                # to perform ADF test
import statsmodels.api as sm                                                  # for various statistical models
import pmdarima as pm                                                         # for auto ARIMA model
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch             # diagnostic tests
from arch import arch_model                                                   # for GARCH model
from statsmodels.tsa.holtwinters import ExponentialSmoothing                  # for Holt's method model

# Machine learning
import tensorflow as tf                                                       # for deep learning
from scipy.stats import randint, uniform                                      # for distribution used in RandomizedSearchCY
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score # to assess models' performance
from sklearn.preprocessing import StandardScaler                              # for data scaling
from keras.models import Sequential                                           # to build neural network models
from keras.layers import LSTM, Dense, Dropout                                 # for LSTM networks
from tensorflow.keras.optimizers import Adam                                  # for model optimization
from scikeras.wrappers import KerasRegressor                                  # to integrate Keras with scikit-learn
from sklearn.model_selection import RandomizedSearchCV                        # for hyperparameter tuning

In [ ]:
tf.keras.utils.set_random_seed(15) # ensure reproducibility of LSTM results

# Section 1: Data Pre-processing

This section shows importing necessary packages, and data importation and cleaning.

## Section 1.1: Import Data

In [ ]:
dataset = pd.read_csv('C:\Desktop\smr20_2000-2024.csv')            # read SMR20 dataset 
dataset['Date'] = pd.to_datetime(dataset['Date'])                  # convert Date column to datetime
dataset.head()                                                     # show the first 5 rows of the data for review

## Section 1.2: Data Cleaning

In [ ]:
dataset.info()                                                  # print information summary of the dataset 
null = dataset.isnull().sum()                                   # find the total no. of missing values in each column
df_null = pd.DataFrame(data = null, columns = ['No. of Null'])  # create a dataframe to show the number of null
print('\n\n', df_null)                                          # number of null in each column is shown
print(f'\n\nThe total no. of null is  {sum(null)}')             # the total number of null is shown

# Section 2: Exploratory Data Analysis

## Section 2.1: Descriptive Statistics

In [ ]:
# Determine the summary statistics of the date column
dataset.Date.describe()

In [ ]:
# Determine the summary statistics of dataset (by default numerical columns)
dataset.describe()

In [ ]:
# Determine the minimum, maximum, average and standard deviation of each numerical column in each year
for col in dataset.select_dtypes(exclude=['datetime64[ns]']).columns:
 desc_stat = dataset.groupby(dataset.Date.dt.year)[[col]].agg(['min','max','mean','std'])
 print(f'\nDescriptive Statistics of:{desc_stat}')
 print("\n")

## Section 2.2: Data Visualization

In [ ]:
# Plot the price over certain number of periods
plt.figure(figsize = (20,8))
plt.plot(dataset['Date'],dataset['Price'],label='Price')
plt.legend(loc=0)
plt.title('SMR20 Historical Price')
plt.show()

In [ ]:
# Plot the actual price and its rolling mean over certain number of previous periods
plt.figure(figsize = (20,8))
plt.plot(dataset['Date'],dataset['Price'],label='Price')
plt.plot(dataset['Date'],dataset['Price'].rolling(30).mean(),label='Rolling Mean (n=30)')
plt.legend(loc=0)
plt.title('SMR20 Historical Price')
plt.show()

In [ ]:
fig, ax= plt.subplots(1,2,figsize=(20, 8))
# Autocorrelation plot
fig=tsaplots.plot_acf(dataset['Price'], lags=72, alpha=0.05, ax=ax[0])
# Partial autocorrelation plot
fig=tsaplots.plot_pacf(dataset['Price'], lags=72,  alpha=0.05, ax=ax[1])
for i in ax.flat:
   i.set(xlabel='Lag at k', ylabel='Correlation coefficient')
plt.show()

# Section 3: Data Splitting and Feature Scaling

In [ ]:
# Set Date column as index
dataset.set_index('Date', inplace=True)

In [ ]:
# Ensure the data is sorted by Date
dataset.sort_index(inplace=True)
dataset

In [ ]:
# Define the proportion of data to use for training
train_size = 0.9  # 90% of the data for training

# Calculate the index at which to split the data
split_index = int(len(dataset) * train_size)

# Split the data into training and testing sets
train, test = dataset[:split_index], dataset[split_index:]

# check the shape of train and test
train.shape, test.shape

In [ ]:
stat_train = train.copy()
stat_test = test.copy()

In [ ]:
ml_train = train.copy()
ml_test = test.copy()

In [ ]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both the training and testing data
train_data = scaler.fit_transform(ml_train)
test_data = scaler.transform(ml_test)

# Section 4: Analysis

## Section 4.1: ARIMA Model

In [ ]:
result = adfuller(stat_train.dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

In [ ]:
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

# Original Series
fig, axes = plt.subplots(2, 2, figsize=(20, 8))
axes[0, 0].plot(stat_train); axes[0, 0].set_title('Original Series')
plot_acf(stat_train, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(stat_train.diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(stat_train.diff().dropna(), ax=axes[1, 1])

plt.show()

In [ ]:
result = adfuller(stat_train.diff().dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(stat_train.diff().dropna(),lags=40,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(stat_train.diff().dropna(),lags=40,ax=ax2)

In [ ]:
model = sm.tsa.ARIMA(stat_train, order=(1,1,0)).fit()
print(model.summary())

In [ ]:
pmd_model = pm.auto_arima(stat_train, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(pmd_model.summary())

In [ ]:
pmd_model.plot_diagnostics(figsize = (15, 10))
plt.show()

In [ ]:
# Compute ACF for the residuals
residuals = pd.DataFrame(model.resid)
acf = sm.tsa.acf(residuals)

# Plot ACF
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plot_acf(residuals, lags=20, ax=plt.gca())  # Adjust 'lags' as needed
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF) of Residuals')

# Plot PACF
plt.subplot(1, 2, 2)
plot_pacf(residuals, lags=20, ax=plt.gca())
plt.xlabel('Lag')
plt.ylabel('Partial Autocorrelation')
plt.title('Partial Autocorrelation Function (PACF) of Residuals')

plt.tight_layout()
plt.show()

In [ ]:
white_noise_arima = acorr_ljungbox(residuals, lags = [10], return_df=True)
white_noise_arima

In [ ]:
LM_pvalue = het_arch(residuals, ddof = 4)[1]
print('LM-test-Pvalue:', '{:.5f}'.format(LM_pvalue))

In [ ]:
mdl_garch = arch_model(residuals, vol = 'GARCH', p = 1, q = 1)
res_fit = mdl_garch.fit()
print(res_fit.summary())

In [ ]:
garch_fit = res_fit

garch_std_resid = pd.Series(garch_fit.resid / garch_fit.conditional_volatility)
fig = plt.figure(figsize=(15, 8))

# Residual
garch_std_resid.plot(ax=fig.add_subplot(3, 1, 1), title='GARCH Standardized-Residual', legend=False)

# ACF/PACF
tsaplots.plot_acf(garch_std_resid, zero=False, lags=40, ax=fig.add_subplot(3, 2, 3))
tsaplots.plot_pacf(garch_std_resid, zero=False, lags=40, ax=fig.add_subplot(3, 2, 4))

# QQ-Plot & Norm-Dist
sm.qqplot(garch_std_resid, line='s', ax=fig.add_subplot(3, 2, 5))
plt.title("QQ Plot")
fig.add_subplot(3, 2, 6).hist(garch_std_resid, bins=40)
plt.title("Histogram")

plt.tight_layout()
plt.show()

In [ ]:
white_noise_garch = acorr_ljungbox(garch_std_resid, lags = [10], return_df=True)
white_noise_garch

In [ ]:
LM_pvalue = het_arch(garch_std_resid, ddof = 4)[1]
print('LM-test-Pvalue:', '{:.5f}'.format(LM_pvalue))

In [ ]:
forecast_train = model.predict(start = stat_train.index[1], end = stat_train.index[-1]) #When d=1 the first residual is nonsense
forecast_test = model.predict(start = len(stat_train), end = len(dataset)-1) 

In [ ]:
# Use GARCH to predict the residual
garch_forecast = garch_fit.forecast(horizon=1)
predicted_et = garch_forecast.mean['h.1'].iloc[-1]
# Combine both models' output: yt = mu + et
train_prediction = forecast_train + predicted_et
test_prediction = forecast_test + predicted_et

In [ ]:
stat_train.loc[:, 'ARIMA-GARCH Forecast'] = train_prediction
stat_train

In [ ]:
stat_test.loc[:, 'ARIMA-GARCH Forecast'] = test_prediction
stat_test

In [ ]:
plt.figure(figsize=(14,7))

plt.plot(dataset.index[:len(stat_train)], dataset['Price'][:len(stat_train)], color='blue', label='Actual Train Price')
plt.plot(stat_train.index, stat_train['ARIMA-GARCH Forecast'], color='red', label='Predicted Train Price')

plt.title('Training Data and Predicted Values of ARIMA-GARCH Model')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# Get test index after making predictions for the test data
test_index = dataset.index[-len(test_prediction):]

plt.figure(figsize=(14, 7))
plt.plot(test_index, stat_test['Price'], color='blue', label='Actual Test Price')
plt.plot(test_index, stat_test['ARIMA-GARCH Forecast'], color='red', label='Predicted Test Price')
plt.title('Test Data and Predicted Values of ARIMA-GARCH Model')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(14,7))

plt.plot(dataset['Price'], color='blue', label='Actual Price')
plt.plot(stat_test['ARIMA-GARCH Forecast'], color='red', label='Predicted Test Price')
plt.title('Overall Data with Predicted Values for Test Data of ARIMA-GARCH Model')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# Calculate evaluation metrics
train_rmse = np.sqrt(mean_squared_error(stat_train['Price'].iloc[1:], train_prediction))
train_mae = mean_absolute_error(stat_train['Price'].iloc[1:], train_prediction)
train_mape = np.mean(np.abs((stat_train['Price'].iloc[1:] - train_prediction) / stat_train['Price'])) * 100
train_r2 = r2_score(stat_train['Price'].iloc[1:], train_prediction)

In [ ]:
test_rmse = np.sqrt(mean_squared_error(stat_test['Price'], test_prediction))
test_mae = mean_absolute_error(stat_test['Price'], test_prediction)
test_mape = np.mean(np.abs((stat_test['Price'] - test_prediction) / stat_test['Price'])) * 100
test_r2 = r2_score(stat_test['Price'], test_prediction)

In [ ]:
# Organize evaluation metrics into a DataFrame
metrics_data = {
    'Metric': ['RMSE', 'MAE', 'MAPE', 'R2 Score'],
    'Train': [train_rmse, train_mae, train_mape, train_r2],
    'Test': [test_rmse, test_mae, test_mape, test_r2]
}

metrics_df = pd.DataFrame(metrics_data)
print("\nEvaluation Metrics:")
print(metrics_df)

## Section 4.2: Double Exponential Smoothing Model

In [ ]:
# Apply Double Exponential Smoothing (Holt's method)
DES_model = ExponentialSmoothing(stat_train['Price'], trend='add')
result = DES_model.fit()

In [ ]:
# Forecast future prices
forecast_train = result.predict(start=stat_train.index[0], end=stat_train.index[-1])  # Forecasting 12 months ahead
forecast_test = result.forecast(steps=len(stat_test))  # Forecasting 12 months ahead

In [ ]:
plt.figure(figsize=(14,7))

plt.plot(dataset.index[:len(stat_train)], dataset['Price'][:len(stat_train)], color='blue', label='Actual Train Price')
plt.plot(stat_train.index, forecast_train, color='red', label='Predicted Train Price')

plt.title('Training Data and Predicted Values of DES Model')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# Get test index after making predictions for the test data
test_index = dataset.index[-len(forecast_test):]

plt.figure(figsize=(14, 7))
plt.plot(test_index, stat_test['Price'], color='blue', label='Actual Test Price')
plt.plot(test_index, forecast_test, color='red', label='Predicted Test Price')
plt.title('Test Data and Predicted Values of DES Model')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# Plot forecast
plt.figure(figsize=(14, 7))
plt.plot(dataset, color='blue', label='Actual Price')
plt.plot(forecast_test, color='red', label='Predicted Test Price')
plt.title('Overall Data with Predicted Values for Test Data of DES Model')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# Calculate evaluation metrics
train_rmse = np.sqrt(mean_squared_error(stat_train['Price'], forecast_train))
train_mae = mean_absolute_error(stat_train['Price'], forecast_train)
train_mape = np.mean(np.abs((stat_train['Price'] - forecast_train) / stat_train['Price'])) * 100
train_r2 = r2_score(stat_train['Price'], forecast_train)

In [ ]:
test_rmse = np.sqrt(mean_squared_error(stat_test['Price'], forecast_test))
test_mae = mean_absolute_error(stat_test['Price'], forecast_test)
test_mape = np.mean(np.abs((stat_test['Price'] - forecast_test) / stat_test['Price'])) * 100
test_r2 = r2_score(stat_test['Price'], forecast_test)

In [ ]:
# Organize evaluation metrics into a DataFrame
metrics_data = {
    'Metric': ['RMSE', 'MAE', 'MAPE', 'R2 Score'],
    'Train': [train_rmse, train_mae, train_mape, train_r2],
    'Test': [test_rmse, test_mae, test_mape, test_r2]
}

metrics_df = pd.DataFrame(metrics_data)
print("\nEvaluation Metrics:")
print(metrics_df)

## Section 4.3: LSTM Model

In [ ]:
# Function to create sequences of data
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length), 0])
        y.append(data[i + seq_length, 0])
    return np.array(X), np.array(y)

In [ ]:
seq_length = 10

In [ ]:
# Generate sequences for training and testing
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

In [ ]:
# Define a function to create and compile the LSTM model
def create_model(units=50, learning_rate=0.001, dropout_rate=0.2, batch_size=32,epochs=100):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(seq_length, 1)))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units))
    model.add(Dense(units=1))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

In [ ]:
# Define parameter distribution for randomized search
param_dist = {
    'model__units': randint(50, 150),               # Randomly select numbers of LSTM units
    'model__learning_rate': uniform(0.0001, 0.01),  # Randomly select learning rates in range [0.0001, 0.01]
    'model__epochs': [50, 100, 150],                # Select specific numbers of epochs
    'model__dropout_rate': [0.1, 0.2, 0.3],         # Select specific dropout rates
    'model__batch_size': [16, 32, 64]               # Select specific batch sizes
}

In [ ]:
# Create the LSTM model
lstm_model = KerasRegressor(model=create_model, verbose=0)

In [ ]:
# Perform randomized search
random_search = RandomizedSearchCV(estimator=lstm_model, param_distributions=param_dist, n_iter=10, scoring='neg_mean_squared_error', cv=3)
random_search_result = random_search.fit(X_train, y_train)

In [ ]:
# Print the best parameters and score
print("Best Parameters:", random_search_result.best_params_)
print("Best Score:", -random_search_result.best_score_)

In [ ]:
# Store the best paramater
best_units = random_search_result.best_params_['model__units']
best_learning_rate = random_search_result.best_params_['model__learning_rate']
best_epochs = random_search_result.best_params_['model__epochs']
best_dropout_rate = random_search_result.best_params_['model__dropout_rate']
best_batch_size = random_search_result.best_params_['model__batch_size']

In [ ]:
# Use the best parameters to create and train the final model
final_model = create_model(units=best_units, learning_rate=best_learning_rate, 
                           dropout_rate=best_dropout_rate, batch_size=best_batch_size)
history = final_model.fit(X_train, y_train, epochs=best_epochs, batch_size=best_batch_size, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# Plot training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Make predictions
train_predictions = final_model.predict(X_train)
test_predictions = final_model.predict(X_test)

In [ ]:
# Inverse transform the predictions
train_predictions = scaler.inverse_transform(train_predictions)
y_train = scaler.inverse_transform([y_train])
test_predictions = scaler.inverse_transform(test_predictions)
y_test = scaler.inverse_transform([y_test])

In [ ]:
# Step 6: Visualize the results
plt.figure(figsize=(14, 7))

# Plotting training data
plt.plot(dataset.index[seq_length:seq_length+len(train_predictions)], y_train[0], color='blue', label='Actual Train Price')
plt.plot(dataset.index[seq_length:seq_length+len(train_predictions)], train_predictions[:, 0], color='red', label='Predicted Train Price')

plt.title('Training Data and Predicted Values of LSTM Model')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# Get test index after making predictions for the test data
test_index = dataset.index[-len(test_predictions):]

# Third Graph: Test Data with Predicted Values
plt.figure(figsize=(14, 7))
plt.plot(test_index, y_test[0], color='blue', label='Actual Test Price')
plt.plot(test_index, test_predictions[:, 0], color='red', label='Predicted Test Price')
plt.title('Test Data and Predicted Values of LSTM Model')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# Second Graph: Overall Data with Predicted Values for Test Data
plt.figure(figsize=(14, 7))
plt.plot(dataset.index, dataset['Price'], color='blue', label='Actual Price')
plt.plot(test_index, test_predictions[:, 0], color='red', label='Predicted Test Price')
plt.title('Overall Data with Predicted Values for Test Data of LSTM Model')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# Calculate evaluation metrics
train_rmse = np.sqrt(mean_squared_error(y_train[0], train_predictions[:, 0]))
train_mae = mean_absolute_error(y_train[0], train_predictions[:, 0])
train_mape = np.mean(np.abs((y_train[0] - train_predictions[:, 0]) / y_train[0])) * 100
train_r2 = r2_score(y_train[0], train_predictions[:, 0])

In [ ]:
test_rmse = np.sqrt(mean_squared_error(y_test[0], test_predictions[:, 0]))
test_mae = mean_absolute_error(y_test[0], test_predictions[:, 0])
test_mape = np.mean(np.abs((y_test[0] - test_predictions[:, 0]) / y_test[0])) * 100
test_r2 = r2_score(y_test[0], test_predictions[:, 0])

In [ ]:
# Organize evaluation metrics into a DataFrame
metrics_data = {
    'Metric': ['RMSE', 'MAE', 'MAPE', 'R2 Score'],
    'Train': [train_rmse, train_mae, train_mape, train_r2],
    'Test': [test_rmse, test_mae, test_mape, test_r2]
}

metrics_df = pd.DataFrame(metrics_data)
print("\nEvaluation Metrics:")
print(metrics_df)